In [1]:
!pip install -qU langchain langchain-openai langgraph>0.2.27

# Build a Chatbot

In this session, we will implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

## Setup

In [ ]:
import os

langchain_api_key = 'your_langchain_api_key_here'  # Replace with your actual LangChain API key
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

## Quickstart

In [ ]:
openai_api_key = 'your_openai_api_key_here'  # Replace with your actual OpenAI API key
os.environ['OPENAI_API_KEY'] = openai_api_key

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-3.5-turbo')

First we use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them.

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content='Hi! My name is Bin')])

AIMessage(content='Hello Bin! Nice to meet you. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 13, 'total_tokens': 28, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3f6e6ee8-e87a-49c8-8a08-dec366fe6b99-0', usage_metadata={'input_tokens': 13, 'output_tokens': 15, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In this case, the model on its own does not have any concept of state. If we ask a followup question:

In [5]:
model.invoke([HumanMessage(content='What is my name?')])

AIMessage(content="I'm sorry, I do not have access to personal information such as your name.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 12, 'total_tokens': 29, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-97bf07a6-c1ce-4729-b21f-1149e849288b-0', usage_metadata={'input_tokens': 12, 'output_tokens': 17, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

We can see that if does not take the previous conversation turn into context, and cannot answer the question.

To get around this, we need to pass the entire conversation history into the model.

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content='Hi! My name is Bin'),
        AIMessage(content='Hi Bin! How can I help you today?'),
        HumanMessage(content='What is my name?'),
    ]
)

AIMessage(content='Your name is Bin.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 36, 'total_tokens': 41, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-57fb39e5-0e51-4c74-95e9-dad33b45c9ae-0', usage_metadata={'input_tokens': 36, 'output_tokens': 5, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

and now we can see that we get a good response.

## Message persistence

**LangGraph** implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

LangGraph comes with a simple in-memory checkpointer, which we use below:

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state['messages'])
    return {'messages': response}


# Define the (single) node in the graph
workflow.add_edge(START, 'model')
workflow.add_node('model', call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

We now need to create a `config` that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a `thread_id`

In [8]:
config = {'configurable': {'thread_id': 'abc123'}}

This enables us to support multiple conversation threads with a single application, a common requirement when our application has multiple users.

Now we can invoke the application:

In [9]:
query = "Hi! My name is Bin."

input_messages = [HumanMessage(query)]

output = app.invoke({'messages': input_messages}, config)
output['messages'][-1].pretty_print() # contains all messages in state

================================== Ai Message ==================================

Hello Bin! How can I assist you today?


In [10]:
query2 = "What is my name?"

input_messages = [HumanMessage(query2)]

output = app.invoke({'messages': input_messages}, config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bin.


Our chatbot now remembers things about us. If we change the config to reference a different `thread_id`, we can see that it starts the conversation fresh:

In [11]:
config2 = {'configurable': {'thread_id': 'xyz456'}}

input_messages = [HumanMessage(query2)]

output = app.invoke({'messages': input_messages}, config2)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, I do not have access to personal information such as your name.


However, we can always go back to the original conversation (since we are persisting it in a database):

In [12]:
output = app.invoke(
    {'messages': input_messages},
    config, # swithc back to our first configuration
)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bin.


For async support, update the `call_model` node to be an async function and use `.ainvoke` when invoking the application:
```python
# Async function for code
async def call_model(state: MessagesState):
    response = await model.ainvoke(state['messages'])
    return {'messages': reponse}

# Define graph as before:
workflow = StateGraph(state_scheme=MessagesState)
workflow.add_edge(START, 'model')
workflow.add_node('model', call_model)
app = workflow.complie(checkpointer=MemorySaver())

# Async invocation
output = await app.ainvoke({'messages': input_messages}, config)
output['messages'][-1].pretty_print()
```

## Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with.

To add in a system message, we will create a `ChatPromptTemplate`. We will utilize `MessagesPlaceholder` to pass all the messages in.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            'You talk like a pirate. Answer all questions to the best of your ability.',
        ),
        MessagesPlaceholder(variable_name='messages')
    ]
)

We can now update our application to incorporate this template:

In [14]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {'messages': response}


workflow.add_edge(START, 'model')
workflow.add_node('model', call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [15]:
config3 = {'configurable': {'thread_id': 'xyz456'}}

query3 = "Hi! I'm Bin."

input_messages = [HumanMessage(query3)]

output = app.invoke({'messages': input_messages}, config3)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Ahoy, Bin! What be ye needin' help with today, matey?


In [16]:
query4 = "What is my name?"

input_messages = [HumanMessage(query4)]

output = app.invoke({'messages': input_messages}, config3)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Bin, me hearty! A fine name for a swashbuckling pirate like yerself. Arrr!


Let's now make our prompt more complicated. Let's assume that the prompt template now looks like this:

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system',
         'You are a helpful assistant. Answer all questions to the best of your ability in {language}.'),
        MessagesPlaceholder(variable_name='messages'),
    ]
)

We have added a new `language` input to the input. Our application has two parameters now -- the input `messages` and `language`.

In [18]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {'messages': [response]}


workflow.add_edge(START, 'model')
workflow.add_node('model', call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [20]:
config = {'configurable': {'thread_id': 'acb456'}}
query5 = "Hi! I'm Bin."
language = 'Spanish'

input_messages = [HumanMessage(query5)]

output = app.invoke(
    {'messages': input_messages,
     'language': language},
    config,
)

output['messages'][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bin! ¿En qué puedo ayudarte hoy?


Note that the entire state is persisted, so we can omit parameters like `language` if no changes are desired:

In [21]:
query6 = "What is my name?"

input_messages = [HumanMessage(query6)]

output = app.invoke({'messages': input_messages}, config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bin.


## Managing conversation history

If we leave the conversation history unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages we are passing in.

**Importantly, we will want to do this BEFORE the prompt template but AFTER we load previous messages from Message Histor.**

In this case, we will use the `trim_messages` helper to reduce how many messages we are sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages:

In [26]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]


trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

To user it in our chain, we just need to run the trimmer before we pass the messages input to our prompt

In [27]:
workflow = StateGraph(state_schema=State)

def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state['messages'])
    response = chain.invoke(
        {'messages': trimmed_messages,
         'language': state['language']},
    )
    return {'messages': [response]}


workflow.add_edge(START, 'model')
workflow.add_node('model', call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

If we try asking the model our name, it will NOT know it since we trimmed that part of the chat history:

In [24]:
config = {'configurable': {'thread_id': 'acb456'}}
query6 = "what is my name?"
language = 'English'

input_messages = messages + [HumanMessage(query6)]

output = app.invoke(
    {'messages': input_messages,
     'language': language},
    config,
)

output['messages'][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, I don't have access to personal information. How may I assist you today?


But if we ask about information that is within the last few messages, it remembers:

In [28]:
config = {'configurable': {'thread_id': 'acb456'}}
query6 = "What math problem did I ask?"
language = 'English'

input_messages = messages + [HumanMessage(query6)]

output = app.invoke(
    {'messages': input_messages,
     'language': language},
    config,
)

output['messages'][-1].pretty_print()

================================== Ai Message ==================================

You asked "what's 2 + 2?"


## Streaming

LLMs can sometimes take a while to respond, and so in order to improve user experience one thing that most applications do is stream back each token as it is generated. This allows the users to see progress.

By default, `.stream` in our LangGraph application streams application steps -- in this case, the single step of the model response. Setting `stream_mode="messages"` allows use to stream output tokens instead:

In [29]:
config = {'configurable': {'thread_id': 'acb789'}}
query = "Hi! I'm Bin. Tell me a joke."
language = 'English'

input_messages = [HumanMessage(query)]

for chunk, metadata in app.stream(
    {'messages': input_messages, 'language': language},
    config,
    stream_mode='messages',
):
    if isinstance(chunk, AIMessage): # filter to just model responses
        print(chunk.content, end='|')

|Sure|,| Bin|!| Here|'s| a| joke| for| you|:

|Why| couldn|'t| the| bicycle| find| its| way| home|?

|Because| it| lost| its| bearings|!| 😄||